# 📘 Ejercicio 2 — Cifrado de Hill

Considere el cifrado de Hill sobre $\mathbb{Z}_{26}$ con bloques de tamaño $2$,
definido por:

$$
E_K(x) = xK \pmod{26}
$$

donde $x \in \mathbb{Z}_{26}^2$ es un vector fila y  
$K \in \mathbb{Z}_{26}^{2\times 2}$ es la matriz clave.

Sea la matriz clave:

$$
K =
\begin{pmatrix}
24 & 5 \\
13 & 7
\end{pmatrix}
$$

## a) ¿La matriz \(K\) constituye una clave válida para el cifrado de Hill?

Para que una matriz \(K\) sea una clave válida en el cifrado de Hill sobre
$\mathbb{Z}_{26}$, es necesario y suficiente que \(K\) sea **invertible módulo 26**.

Esto equivale a exigir que su determinante sea coprimo con 26, es decir:

$$
\gcd(\det(K), 26) = 1
$$

Calculamos el determinante de la matriz \(K\):

$$
\det(K) = (24)(7) - (5)(13)
$$

$$
\det(K) = 168 - 65 = 103
$$

Reducimos el determinante módulo 26:

$$
103 \equiv 25 \pmod{26}
$$

Ahora verificamos la coprimalidad:

$$
\gcd(25,26) = 1
$$

Como el determinante de \(K\) es coprimo con 26, la matriz \(K\) es invertible
módulo 26.

Por lo tanto, se concluye que:

$$
\boxed{\text{La matriz } K \text{ constituye una clave válida para el cifrado de Hill sobre } \mathbb{Z}_{26}.}
$$

In [10]:
import numpy as np

M = 26  # módulo

# ---- mismas utilidades (euclid + inverso) estilo ejercicio 1 ----
def euclid(a: int, b: int) -> int:
    a, b = abs(a), abs(b)
    while b != 0:
        a, b = b, a % b
    return a

def egcd(a: int, b: int):
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return (g, x, y)

def inverso(a: int, m: int) -> int:
    g, x, _ = egcd(a, m)
    if g != 1:
        raise ValueError(f"No existe inverso: gcd({a},{m})={g}")
    return x % m

# ---- matriz clave dada ----
K = np.array([[24, 5],
              [13, 7]], dtype=int)

# ---- determinante entero de una 2x2: ad - bc ----
a, b = K[0,0], K[0,1]
c, d = K[1,0], K[1,1]
detK = a*d - b*c
detK_mod = detK % M

print("Matriz K:\n", K)
print("\nDet(K) =", detK)
print("Det(K) mod 26 =", detK_mod)
print("gcd(Det(K), 26) =", euclid(detK_mod, M))

# ---- verificación de validez (invertible mod 26) ----
if euclid(detK_mod, M) != 1:
    print("\n❌ K NO es clave válida (no es invertible módulo 26).")
else:
    print("\n✅ K es clave válida (es invertible módulo 26).")


Matriz K:
 [[24  5]
 [13  7]]

Det(K) = 103
Det(K) mod 26 = 25
gcd(Det(K), 26) = 1

✅ K es clave válida (es invertible módulo 26).


## b) Descifrado de los textos interceptados

Se interceptaron los siguientes bloques cifrados (tamaño 2), generados con la matriz clave:

$$
K=
\begin{pmatrix}
24 & 5\\
13 & 7
\end{pmatrix}
$$

Los textos cifrados interceptados son:

$$
c=(M,L), \qquad c'=(R,E)
$$

Usando la codificación $A=0,\ldots,Z=25$ y la regla de descifrado del cifrado de Hill:

$$
m \equiv cK^{-1} \pmod{26},
$$

se obtiene:

- Para $c=(M,L)$, el bloque en claro es:

$$
m = (H,E)
$$

- Para $c'=(R,E)$, el bloque en claro es:

$$
m' = (L,P)
$$

Por lo tanto, los textos descifrados son:

$$
c=(M,L)\ \longrightarrow\ \textbf{HE},
\qquad
c'=(R,E)\ \longrightarrow\ \textbf{LP}.
$$

In [11]:
import string
ALFABETO = string.ascii_uppercase
M = 26

def normalizar(texto: str) -> str:
    return "".join([c for c in texto.upper() if c in ALFABETO])

def inv_matriz_mod_np(K, mod=26):
    # Parche: para 2x2 usar fórmula cerrada (siempre funciona cuando det es invertible)
    K = np.array(K, dtype=int) % mod
    n = K.shape[0]
    if K.shape[1] != n:
        raise ValueError("K debe ser cuadrada (j×j).")

    if n != 2:
        raise ValueError("Este parche solo implementa la inversa para 2x2 (como pide el ejercicio).")

    a, b = int(K[0,0]), int(K[0,1])
    c, d = int(K[1,0]), int(K[1,1])

    det = (a*d - b*c) % mod
    if euclid(det, mod) != 1:
        raise ValueError("La matriz no es invertible mod 26.")

    det_inv = inverso(det, mod)  # usa TU función inverso(a, m)

    adj = np.array([[ d, -b],
                    [-c,  a]], dtype=int) % mod

    return (det_inv * adj) % mod


def hill_cifrar(texto: str, K):
    K = np.array(K, dtype=int) % M
    j = K.shape[0]

    P = normalizar(texto)
    P += "X" * ((-len(P)) % j)   # padding a múltiplo de j

    nums = np.array([ord(c) - 65 for c in P], dtype=int)
    bloques = nums.reshape(-1, j)      # vectores fila
    cif = (bloques @ K) % M            # c = mK

    return "".join(chr(x + 65) for x in cif.reshape(-1))

def hill_descifrar(texto_cifrado: str, K):
    K = np.array(K, dtype=int) % M
    j = K.shape[0]

    C = normalizar(texto_cifrado)
    if len(C) % j != 0:
        raise ValueError("Longitud del cifrado no es múltiplo de j.")

    K_inv = inv_matriz_mod_np(K, M)

    nums = np.array([ord(c) - 65 for c in C], dtype=int)
    bloques = nums.reshape(-1, j)
    pla = (bloques @ K_inv) % M         # m = cK^{-1}

    return "".join(chr(x + 65) for x in pla.reshape(-1))


In [12]:
K = [[24, 5],
     [13, 7]]

c  = "ML"   # (M,L)
cp = "RE"   # (R,E)

print("K_inv mod 26:\n", inv_matriz_mod_np(K, 26))

m  = hill_descifrar(c, K)
mp = hill_descifrar(cp, K)

print("c  =", (c[0], c[1]), "->", m)
print("c' =", (cp[0], cp[1]), "->", mp)

K_inv mod 26:
 [[19  5]
 [13  2]]
c  = ('M', 'L') -> HE
c' = ('R', 'E') -> LP


## c) Ataque de texto claro conocido

Se considera un escenario de **ataque de texto claro conocido**, en el cual el
atacante dispone de pares \((m_i, c_i)\) (texto claro, texto cifrado), pero
desconoce la matriz clave \(K\). Cada par satisface:

$$
c_i \equiv m_i K \pmod{26},
$$

donde \(m_i \in \mathbb{Z}_{26}^{1\times 2}\) y \(c_i \in \mathbb{Z}_{26}^{1\times 2}\).

---

### a) Modelación matemática del problema

Si el atacante dispone de \(t\) pares conocidos \((m_i, c_i)\), estos pueden
agruparse en matrices:

$$
M =
\begin{pmatrix}
m_1 \\
m_2 \\
\vdots \\
m_t
\end{pmatrix}
\in \mathbb{Z}_{26}^{t\times 2},
\qquad
C =
\begin{pmatrix}
c_1 \\
c_2 \\
\vdots \\
c_t
\end{pmatrix}
\in \mathbb{Z}_{26}^{t\times 2}.
$$

El sistema completo queda modelado como:

$$
C \equiv M K \pmod{26}.
$$

Por lo tanto, el problema de recuperar la matriz \(K\) se reduce a resolver un
**sistema lineal modular**. Si existe una submatriz \(2\times 2\) de \(M\) que sea
invertible módulo 26, entonces la matriz clave se obtiene como:

$$
K \equiv M^{-1} C \pmod{26}.
$$

---

### b) Número mínimo de pares necesarios y condición

La matriz \(K\) es de tamaño \(2\times 2\), por lo que contiene **cuatro
incógnitas**. Cada par \((m_i, c_i)\) proporciona **dos ecuaciones lineales**,
una por cada componente del vector \(c_i\). En consecuencia, el número mínimo de
pares necesarios es:

$$
\boxed{2 \text{ pares } (m_i, c_i)}.
$$

Estos dos pares deben satisfacer una condición adicional: los vectores de texto
claro \(m_1\) y \(m_2\) deben ser **linealmente independientes** en
\(\mathbb{Z}_{26}^2\). Equivalentemente, la matriz

$$
M =
\begin{pmatrix}
m_1 \\
m_2
\end{pmatrix}
$$

debe ser invertible módulo 26, lo que ocurre si y solo si:

$$
\boxed{\gcd(\det(M), 26) = 1}.
$$

Cuando esta condición se cumple, la matriz clave \(K\) puede recuperarse de forma
única mediante:

$$
K \equiv M^{-1} C \pmod{26}.
$$